## quickstart compare runs,choose a model and deploy it to a REST API 

also build a container impage suitable for deployment to a cloud platform

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [2]:
# load a dataset
data=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")
data


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
## split data into training validation and test sets
train, test = train_test_split(data, test_size=0.2, random_state=42)
train 

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4665,7.3,0.17,0.36,8.20,0.028,44.0,111.0,0.99272,3.14,0.41,12.4,6
1943,6.3,0.25,0.44,11.60,0.041,48.0,195.0,0.99680,3.18,0.52,9.5,5
3399,5.6,0.32,0.33,7.40,0.037,25.0,95.0,0.99268,3.25,0.49,11.1,6
843,6.9,0.19,0.35,1.70,0.036,33.0,101.0,0.99315,3.21,0.54,10.8,7
2580,7.7,0.30,0.26,18.95,0.053,36.0,174.0,0.99976,3.20,0.50,10.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [4]:
train_x=train.drop(columns=["quality"]).values
train_y=train["quality"].values.ravel()

#test dataset
test_x=test.drop(columns=["quality"]).values
test_y=test["quality"].values.ravel()

# splitting the traing set into training and validation sets
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.2,random_state=42)

signature = infer_signature(train_x, train_y)

In [5]:
## ANN model 

def train_model(params,epochs,train_x,train_y,valid_x,valid_y, test_x, test_y):

    # define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model = keras.models.Sequential([
        keras.Input(shape=(train_x.shape[1],)),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64,activation='relu'),
        keras.layers.Dense(1)
    ])
    # compile the model
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=params['learning_rate'],momentum=params['momentum']),
                  loss='mean_squared_error',
                  metrics=['mean_squared_error'])
    # train the model
    with mlflow.start_run(nested=True) :
        model.fit(train_x, train_y,
                  validation_data=(valid_x, valid_y),
                  epochs=epochs,batch_size=64)
        # evaluate the model
        eval_result = model.evaluate(valid_x,valid_y,batch_size=64)
        eval_rmse = eval_result[1]
        # log parameters, metrics and model
        mlflow.log_param('params',params)
        mlflow.log_metric("valid_rmse", eval_rmse)

        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {
            'loss': eval_rmse,
            'status': STATUS_OK,
            'model': model
        }





In [6]:
def objective(params):
    result= train_model(params, epochs=10, train_x=train_x, train_y=train_y, valid_x=valid_x, valid_y=valid_y,test_x=test_x, test_y=test_y)
    return result

In [7]:
space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(1e-5), np.log(1e-1)),
    'momentum': hp.uniform('momentum', 0.0, 1.0)
}

In [8]:
mlflow.set_experiment("wine_quality_prediction")
with mlflow.start_run():
    # run hyperparameter optimization
    trials = Trials()
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=4, trials=trials)
    best_run = sorted(trials.results, key=lambda x: x['loss'])[0]

    mlflow.log_param('best',best)
    mlflow.log_metric("eval_rmse", best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'], "best_model", signature=signature)
    print("Best hyperparameters:", best)
    print("Best validation RMSE:", best_run['loss'])

Epoch 1/10                                           

 1/49 ━━━━━━━━━━━━━━━━━━━━ 29s 617ms/step - loss: 39.0350 - mean_squared_error: 39.0350
29/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38.5759 - mean_squared_error: 38.5759   
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 38.1166 - mean_squared_error: 38.1166 - val_loss: 36.6229 - val_mean_squared_error: 36.6229

Epoch 2/10                                           

 1/49 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 35.9673 - mean_squared_error: 35.9673
34/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35.6958 - mean_squared_error: 35.6958 
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.4539 - mean_squared_error: 35.4539 - val_loss: 34.0712 - val_mean_squared_error: 34.0712

Epoch 3/10                                           

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 35.7241 - mean_squared_error: 35.7241
34/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33.3381 - mean_squared_error: 33.3381 
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/s

2025/06/30 17:32:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/10                                                                     

 1/49 ━━━━━━━━━━━━━━━━━━━━ 43s 900ms/step - loss: 36.0075 - mean_squared_error: 36.0075
14/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 32.9342 - mean_squared_error: 32.9342   
26/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 30.9932 - mean_squared_error: 30.9932
38/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 29.1859 - mean_squared_error: 29.1859
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 27.4993 - mean_squared_error: 27.4993 - val_loss: 10.7997 - val_mean_squared_error: 10.7997

Epoch 2/10                                                                     

 1/49 ━━━━━━━━━━━━━━━━━━━━ 4s 91ms/step - loss: 11.3430 - mean_squared_error: 11.3430
11/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 9.9446 - mean_squared_error: 9.9446   
25/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.0558 - mean_squared_error: 9.0558
42/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.2622 - mean_squared_error: 8.2622
49/49 ━━━━━━━

2025/06/30 17:32:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/10                                                                     

 1/49 ━━━━━━━━━━━━━━━━━━━━ 46s 960ms/step - loss: 30.7218 - mean_squared_error: 30.7218
13/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 27.6725 - mean_squared_error: 27.6725   
25/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 22.2791 - mean_squared_error: 22.2791
39/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 18.6464 - mean_squared_error: 18.6464
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 16.6578 - mean_squared_error: 16.6578 - val_loss: 3.1252 - val_mean_squared_error: 3.1252

Epoch 2/10                                                                     

 1/49 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - loss: 2.5590 - mean_squared_error: 2.5590
13/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.7821 - mean_squared_error: 2.7821 
27/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.5279 - mean_squared_error: 2.5279
41/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3260 - mean_squared_error: 2.3260
49/49 ━━━━━━━━━━━━━

...

Epoch 1/10                                                                     

 1/49 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - loss: 33.8725 - mean_squared_error: 33.8725
14/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 12.8307 - mean_squared_error: 12.8307
27/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.9957 - mean_squared_error: 8.9957  
41/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.0107 - mean_squared_error: 7.0107
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 6.2045 - mean_squared_error: 6.2045 - val_loss: 0.5265 - val_mean_squared_error: 0.5265

Epoch 2/10                                                                     

 1/49 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - loss: 0.5819 - mean_squared_error: 0.5819
12/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5728 - mean_squared_error: 0.5728 
25/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5703 - mean_squared_error: 0.5703
37/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5661 - mean_squared_error: 0.5661
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s

2025/06/30 17:33:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [01:31<00:00, 22.93s/trial, best loss: 0.48280757665634155]

2025/06/30 17:33:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best hyperparameters: {'learning_rate': np.float64(0.06220329648493852), 'momentum': np.float64(0.7789535649305802)}
Best validation RMSE: 0.48280757665634155


In [13]:
## inference on test set
from mlflow.models import validate_serving_input

model_uri= "runs:/da123f3d2da54f649ac672f6756517f7/best_model"

from mlflow.models import convert_input_example_to_serving_input

serving_payload = convert_input_example_to_serving_input(test_x)
 
validate_serving_input(model_uri,serving_payload)

c:\Users\ashwi\anaconda3\envs\tf310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[5.931267 ],
       [6.5627112],
       [6.335108 ],
       [5.909831 ],
       [6.471781 ],
       [5.829932 ],
       [5.4055514],
       [5.271114 ],
       [6.163168 ],
       [5.5080423],
       [5.92919  ],
       [5.232432 ],
       [6.5877085],
       [5.541965 ],
       [6.4628477],
       [5.4972925],
       [6.036207 ],
       [5.9566154],
       [6.212357 ],
       [5.636814 ],
       [5.6680436],
       [5.8540597],
       [5.536494 ],
       [6.4326115],
       [6.0333624],
       [5.790457 ],
       [5.4294963],
       [6.178778 ],
       [6.077438 ],
       [5.4655714],
       [5.4807076],
       [5.800915 ],
       [5.542795 ],
       [5.7125235],
       [5.657565 ],
       [6.3119183],
       [5.768634 ],
       [5.517171 ],
       [5.725259 ],
       [5.6952376],
       [5.7009435],
       [5.4860096],
       [6.0268292],
       [5.630661 ],
       [5.396512 ],
       [5.6745086],
       [5.63579  ],
       [5.298608 ],
       [5.7150126],
       [5.478783 ],


In [14]:
# load the model as a PyFuncModel
import mlflow.pyfunc
model_uri= "runs:/da123f3d2da54f649ac672f6756517f7/best_model"
loaded_model = mlflow.pyfunc.load_model(model_uri)
# make predictions
predictions = loaded_model.predict(test_x)

 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [15]:
## regestry 
mlflow.register_model(model_uri, "wine_quality_model")

Successfully registered model 'wine_quality_model'.
2025/06/30 17:50:18 WARNING mlflow.tracking._model_registry.fluent: Run with id da123f3d2da54f649ac672f6756517f7 has no artifacts at artifact path 'best_model', registering model based on models:/m-2469cdc6f13f46d88940617fb2359696 instead
Created version '1' of model 'wine_quality_model'.


<ModelVersion: aliases=[], creation_timestamp=1751269818098, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1751269818098, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='eval_rmse', model_id='m-2469cdc6f13f46d88940617fb2359696', run_id='da123f3d2da54f649ac672f6756517f7', step=0, timestamp=1751268811137, value=0.48280757665634155>], model_id='m-2469cdc6f13f46d88940617fb2359696', name='wine_quality_model', params={'best': "{'learning_rate': np.float64(0.06220329648493852), 'momentum': "
         'np.float64(0.7789535649305802)}'}, run_id='da123f3d2da54f649ac672f6756517f7', run_link=None, source='models:/m-2469cdc6f13f46d88940617fb2359696', status='READY', status_message=None, tags={}, user_id=None, version=1>